In [1]:
#!pip install mesa


In [3]:
#Importing the libraries
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid,SingleGrid
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

import numpy as np

In [ ]:
#Creating the agent

class UltimatumAgent(Agent):
    """ An agent with fixed initial wealth and a personality (a,o). See read me for an explanation of the game."""
    def __init__(self, unique_id, model,personality):
        super().__init__(unique_id, model)
        self.wealth = 10
        self.personality=personality #personality is a tuple (a,o)=(accept, offer)
        self.accept=personality[0] #accept at least a
        self.offer=personality[1] #offers o

    def step(self):
        #The agent's step starts here
        if self.wealth <= 0:
            self.model.schedule.remove(self)  #dies if 0
            #print("morto"+str(self.unique_id))
            #print(self.personality)
            return
        
       #select another random agent 
        other_agent = self.random.choice(self.model.schedule.agents)
        
        #if they agree they both get money 
        if(other_agent.accept<=self.offer):
          print(str(other_agent.unique_id)+" gets "+ str(100*self.offer)+" from "+str(self.unique_id))
          other_agent.wealth += 100*self.offer
          self.wealth += 100*(1-self.offer)
        
        #if they don't agree nobody gets money
        print(str(other_agent.unique_id)+" gets nothing from"+str(self.unique_id))

        #print("Steps"+str(self.model.num_steps))
        #here is life expenses what changes if i put this at the beginning? what if I increase to cost with passing of time
        self.wealth -= 1 *self.model.num_steps  

In [6]:
#Creating the game
class UltimatumModel(Model):
    """A model with some number of agents."""
    def __init__(self, N):
        self.num_agents = N
        self.num_steps=0
        self.schedule = RandomActivation(self)
        # Create agents
        #for i in range(self.num_agents):
        temp = np.array([0.1,.2,.3,.4,.5,.6,.7,.8,.9,1])
        y=[[x0, y0] for x0 in temp for y0 in temp]
        for k in range(N):
            a = UltimatumAgent(k, self,y[k])
            self.schedule.add(a)

        self.datacollector = DataCollector(
            #model_reporters={"Gini": compute_gini},  # `compute_gini` defined above
            agent_reporters={"Wealth": "wealth","Personality":"personality"})

    def step(self):
        '''Advance the model by one step.'''
        self.num_steps+=1
        
        self.datacollector.collect(self)
        self.schedule.step()



In [7]:
#Run the basic model
model = MoneyModel(100)
for i in range(100):
    model.step()

Output streaming troncato alle ultime 5000 righe.
50 gets nothing from39
76 gets 80.0 from 67
76 gets nothing from67
50 gets 60.0 from 35
50 gets nothing from35
79 gets 80.0 from 47
79 gets nothing from47
0 gets 10.0 from 20
0 gets nothing from20
79 gets nothing from85
14 gets 40.0 from 23
14 gets nothing from23
62 gets nothing from81
81 gets nothing from26
84 gets nothing from11
31 gets 50.0 from 34
31 gets nothing from34
21 gets 90.0 from 28
21 gets nothing from28
28 gets nothing from1
72 gets nothing from24
48 gets 70.0 from 86
48 gets nothing from86
35 gets 50.0 from 74
35 gets nothing from74
37 gets 80.0 from 17
37 gets nothing from17
7 gets 70.0 from 76
7 gets nothing from76
22 gets 90.0 from 18
22 gets nothing from18
45 gets 50.0 from 4
45 gets nothing from4
88 gets 90.0 from 58
88 gets nothing from58
57 gets 90.0 from 48
57 gets nothing from48
39 gets 40.0 from 83
39 gets nothing from83
62 gets nothing from41
66 gets 70.0 from 6
66 gets nothing from6
51 gets nothing from13
52 g

In [89]:
#Save the data 
agent_wealth = model.datacollector.get_agent_vars_dataframe()
agent_wealth

Wealth Personality
Step AgentID                    
0    0          10.0  [0.1, 0.1]
     1          10.0  [0.1, 0.2]
     2          10.0  [0.1, 0.3]
     3          10.0  [0.1, 0.4]
     4          10.0  [0.1, 0.5]
...              ...         ...
99   66        390.0  [0.7, 0.7]
     67        660.0  [0.7, 0.8]
     73        650.0  [0.8, 0.4]
     76        970.0  [0.8, 0.7]
     83        160.0  [0.9, 0.4]

[8780 rows x 2 columns]